In [13]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [14]:
TRAIN_DIR = "/kaggle/input/emotion-detection-fer/train"
TEST_DIR = "/kaggle/input/emotion-detection-fer/test"

In [15]:
def create_dataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [16]:
train = pd.DataFrame()
train['image'], train['label'] = create_dataframe(TRAIN_DIR)

fearful completed
disgusted completed
angry completed
neutral completed
sad completed
surprised completed
happy completed


In [17]:
test = pd.DataFrame()
test['image'], test['label'] = create_dataframe(TEST_DIR)

fearful completed
disgusted completed
angry completed
neutral completed
sad completed
surprised completed
happy completed


In [18]:
def extract_features(images):
    features = []
    for image in images:
        img = load_img(image, color_mode="grayscale")
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [19]:
train_features = extract_features(train['image']) 

In [20]:
test_features = extract_features(test['image'])

In [21]:
x_train = train_features / 255.0
x_test = test_features / 255.0

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
le = LabelEncoder()
le.fit(train['label'])
y_train = to_categorical(le.transform(train['label']), num_classes=7)
y_test = to_categorical(le.transform(test['label']), num_classes=7)

In [24]:
from keras.layers import BatchNormalization, Activation

model = Sequential()

# First conv layer
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second conv layer
model.add(Conv2D(128, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third conv layer
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fourth conv layer
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
history = model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
  2/225 ━━━━━━━━━━━━━━━━━━━━ 18s 81ms/step - accuracy: 0.1211 - loss: 2.5385   

I0000 00:00:1713197670.585321      89 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713197670.611168      89 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.2174 - loss: 2.0298

W0000 00:00:1713197698.021220      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1713197699.226723      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


225/225 ━━━━━━━━━━━━━━━━━━━━ 59s 138ms/step - accuracy: 0.2176 - loss: 2.0293 - val_accuracy: 0.1785 - val_loss: 2.1557
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 56ms/step - accuracy: 0.3836 - loss: 1.6006 - val_accuracy: 0.3704 - val_loss: 1.6540
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 57ms/step - accuracy: 0.4679 - loss: 1.3740 - val_accuracy: 0.4533 - val_loss: 1.4380
Epoch 4/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 58ms/step - accuracy: 0.5199 - loss: 1.2552 - val_accuracy: 0.5235 - val_loss: 1.2455
Epoch 5/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 59ms/step - accuracy: 0.5579 - loss: 1.1680 - val_accuracy: 0.3764 - val_loss: 1.6281
Epoch 6/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 59ms/step - accuracy: 0.5824 - loss: 1.1121 - val_accuracy: 0.5535 - val_loss: 1.1380
Epoch 7/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.6006 - loss: 1.0576 - val_accuracy: 0.5162 - val_loss: 1.2341
Epoch 8/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.6153 - loss: 1.0263 - val

In [27]:
history = model.fit(x=x_test, y=y_test, batch_size=128, epochs=100, validation_data=(x_train, y_train))

Epoch 1/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 31s 547ms/step - accuracy: 0.6153 - loss: 1.6309 - val_accuracy: 0.9427 - val_loss: 0.3219
Epoch 2/100


W0000 00:00:1713199095.270549      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


57/57 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.6600 - loss: 0.9414 - val_accuracy: 0.9627 - val_loss: 0.2762
Epoch 3/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 141ms/step - accuracy: 0.7126 - loss: 0.8264 - val_accuracy: 0.9899 - val_loss: 0.1786
Epoch 4/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 143ms/step - accuracy: 0.7253 - loss: 0.7414 - val_accuracy: 0.9379 - val_loss: 0.2895
Epoch 5/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 143ms/step - accuracy: 0.7737 - loss: 0.6258 - val_accuracy: 0.9799 - val_loss: 0.1732
Epoch 6/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 140ms/step - accuracy: 0.8074 - loss: 0.5397 - val_accuracy: 0.9042 - val_loss: 0.3308
Epoch 7/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 139ms/step - accuracy: 0.8394 - loss: 0.4472 - val_accuracy: 0.8550 - val_loss: 0.4149
Epoch 8/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 138ms/step - accuracy: 0.8695 - loss: 0.3731 - val_accuracy: 0.9180 - val_loss: 0.2546
Epoch 9/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 137ms/step - accuracy: 0.8853 - loss: 0.3165 - val_accuracy: 0.983

In [28]:
val_acc = history.history['val_accuracy']
print("Validation accuracy:", val_acc[-1])

Validation accuracy: 0.8932738900184631


In [29]:
x_test[0]

array([[[0.32941176],
        [0.31372549],
        [0.32941176],
        ...,
        [0.68235294],
        [0.66666667],
        [0.66666667]],

       [[0.33333333],
        [0.31764706],
        [0.32941176],
        ...,
        [0.6745098 ],
        [0.64705882],
        [0.65490196]],

       [[0.3254902 ],
        [0.32156863],
        [0.34901961],
        ...,
        [0.67843137],
        [0.65098039],
        [0.66666667]],

       ...,

       [[0.11372549],
        [0.12156863],
        [0.16862745],
        ...,
        [0.71764706],
        [0.82352941],
        [0.85490196]],

       [[0.10980392],
        [0.11764706],
        [0.1372549 ],
        ...,
        [0.7372549 ],
        [0.81568627],
        [0.84705882]],

       [[0.08627451],
        [0.10196078],
        [0.12941176],
        ...,
        [0.75294118],
        [0.81568627],
        [0.84313725]]])

In [30]:
probs = model.predict(x_test)

225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step


In [31]:
predicted_labels = np.argmax(probs, axis=1)

In [32]:
max_probs = np.max(probs, axis=1)

In [33]:
predicted_emotions = le.inverse_transform(predicted_labels)

In [34]:
top_predicted_index = np.argmax(max_probs)
top_predicted_emotion = predicted_emotions[top_predicted_index]
top_confidence_score = max_probs[top_predicted_index]

In [35]:
print("Top predicted emotion:", top_predicted_emotion)
print("Confidence score:", top_confidence_score)


Top predicted emotion: fearful
Confidence score: 1.0


In [44]:
model_json = model.to_json()
with open("emotionfacedetect.json", 'w') as json_file:
    json_file.write(model_json)
model.save_weights("emotionfacedetect.weights.h5")

In [45]:
model_json

'{"module": "keras", "class_name": "Sequential", "config": {"name": "sequential", "trainable": true, "dtype": "float32", "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 48, 48, 1], "dtype": "float32", "sparse": false, "name": "input_layer"}, "registered_name": null}, {"module": "keras.layers", "class_name": "Conv2D", "config": {"name": "conv2d", "trainable": true, "dtype": "float32", "filters": 64, "kernel_size": [3, 3], "strides": [1, 1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_con

In [42]:
import joblib
joblib.dump(model_json, 'emotionfacedetect.pkl')

['emotionfacedetect.pkl']